# Organoid Batch Processing Workflow

This notebook runs the workflow demonstrated in [Single_image_workflow.ipynb](Single_image_workflow.ipynb) for all images in a specified folder.

To run this notebook, you need to place all organoid images in a folder named `Organoids` under a `data` folder in the root of this repository (or adjust the path accordingly).

Below is an exampe of the folder structure:

```
Brightfield-3D-models-image-analysis
|
├─ code
|   |
|   ⁞
|   
├─ data
|   |
⁞   ├─ Organoids
    |    |
    |    |
    ⁞    ├─ Patient_1
         |    |
         |    ├─ Day_1
         |    |    |
         |    |    ├─ media1.tif
         |    |    ├─ media2.tif
         ⁞    ⁞    ⁞
```

The conda environment used for this notebook can be created following the instructions in the [README.md](README.md) file. Running this notebook took around 30 seconds in the machine described in the [README.md](README.md) file.

## Import libraries

In [ ]:
import napari_simpleitk_image_processing as nsitk
import pyclesperanto_prototype as cle

from skimage.io import imread, imsave
from pathlib import Path
from napari_skimage_regionprops import regionprops_table

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='skimage') # Ignore low resolution image warnings

%load_ext watermark

## Function with the workflow

Basically, this was created by copy-pasting, in the same order, each command from the workflow notebook.

Then, a generic input argument `image_path` was provided and 2 return variables were added: `image8_eloe` (the final labeled image) and `table`.

If you want to measure more things than just size, take a look at the arguments of the function `regionprops_table` and change them from `False` to `True`. For example, if you want shape measurements, provide an extra argument `shape=True,`.

In [2]:
def workflow(image_path):

    image0_O1 = imread(image_path)
    
    image1_L = nsitk.laplacian_of_gaussian_filter(image0_O1, 1.0)
    
    image2_S = nsitk.standard_deviation_filter(image1_L, 1, 1, 0)
    
    image3_to = cle.threshold_otsu(image2_S)
    
    image4_cl = cle.closing_labels(image3_to, None, 1.0)
    
    image5_B = nsitk.binary_fill_holes(image4_cl)
    
    image6_C = cle.label(image5_B)
    
    image7_esl = cle.exclude_small_labels(image6_C, None, 300.0)
    
    image8_eloe = cle.exclude_labels_on_edges(image7_esl)
    
    table = regionprops_table(
        image0_O1,
        image8_eloe,
        size=True,
        intensity=False,
        napari_viewer=None,
    )
    
    return image8_eloe, table

## Path to the images

Please replace the path below to your local data folder.

In [3]:
data_path =  Path("../../data/Organoids/")

Store image paths to a list of paths.

In [4]:
# Create empty list of paths
image_path_list = []
# iterate over folder path and its subfolders
for path in data_path.glob('**/*.tif'):
    # Append the path to the list
    if not path.stem.endswith('_labels'):
        # Only include images that end with 'O1.tif'
        image_path_list += [path]

Below we display the list of paths.

In [5]:
image_path_list

[WindowsPath('../../data/Organoids/Patient_3/Day_1/Patient 3 organoids_B0687A93A16-5BE0-4369-AC91-702B557BBB1E.tif'),
 WindowsPath('../../data/Organoids/Patient_3/Day_1/Patient 3 organoids_B0821BF40DC-6E41-4C37-904D-4C977F259651.tif'),
 WindowsPath('../../data/Organoids/Patient_3/Day_1/Patient 3 organoids_B110E493636-B580-44C2-9154-49477C2626C6.tif'),
 WindowsPath('../../data/Organoids/Patient_3/Day_1/Patient 3 organoids_B137BAA72B6-2299-4D93-B63A-6E41B7976DA0.tif'),
 WindowsPath('../../data/Organoids/Patient_3/Day_1/Patient 3 organoids_B142FE4EB31-6F7C-4B07-9783-1C44FD2175A5.tif')]

## Batch Processing

For each path in the list, read the image, apply the workflow and save the results in an "Outputs" folder.

Labeled images and tables are saved inside the Outputs folder with the same name as the original image + "\_labels" or "\_table", respectively. 

In [6]:
# iterate over list of paths
for image_path in image_path_list:
    # applies workflow on image
    label_image, table = workflow(image_path)
    
    # Create an empty "Outputs" folder (if not there already)
    output_folder_path = Path(image_path.parent, 'Outputs')
    output_folder_path.mkdir(exist_ok = True)
    
    # Create a "image_labels.tif" path
    output_label_image_path = Path(output_folder_path, image_path.stem + '_labels.tif')
    # Store the label_image in the created "image_labels.tif" path
    imsave(output_label_image_path, label_image)
    
    # Create a "image_table.csv" path
    output_table_path = Path(output_folder_path, image_path.stem + '_table.csv')
    # Store the table in the created "image_table.csv" path
    table.to_csv(output_table_path)

## Version Information

In [7]:
%watermark -v -m --gpu -p napari,numpy,pyclesperanto_prototype,pandas,skimage,napari_simpleitk_image_processing,napari_skimage_regionprops

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.4.0

napari                           : 0.5.0
numpy                            : 2.2.6
pyclesperanto_prototype          : 0.24.4
pandas                           : 2.2.3
skimage                          : 0.25.1
napari_simpleitk_image_processing: 0.4.9
napari_skimage_regionprops       : 0.10.1

Compiler    : MSC v.1943 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 16
Architecture: 64bit

GPU Info: 
  GPU 0: NVIDIA GeForce GTX 1650 Ti with Max-Q Design

